## Preparations

### import modules

In [1]:
import math
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

### Prepare data

#### Prepare train_set

In [2]:
def ToWord(title):
    target_words=[]
    for word in title.split(' '):
        if word != '' and word != ' ':
            target_words.append(word)
        
    return target_words

In [3]:
def PrepareTrainSet():
    with open('Book3.csv','r') as r:
        reader=csv.reader(r)
        index=0
        train_set={}

            
        for row in reader:
            person=[] 
            title_sequence=[]
            #delete the empty element
            for element in row:
                if len(element) == 0:
                    break
                else:
                    person.append(element) #person: id+job_title_sequence
            title_sequence=person[1:]      #title_sequnce: job sequence without id
            max_length=len(title_sequence)-1
            for i in range(1,max_length+1):
                for j in range(0,len(title_sequence)-i):
                    data={}
                    data['input']=[]
                    for k in range(j,i+j):
                        data['input'].append(title_sequence[k])
#                        data['input'].append(str(EOS_token))
                    title=title_sequence[i+j]
                    data['target']=ToWord(title)
#                    data['target'].append(str(EOS_token))
                    train_set[index]=data
                    index += 1
           
    return train_set

In [4]:
train_set = PrepareTrainSet()

In [5]:
lenTrainSet=len(train_set)
lenTrainSet

315174

### Filter the train_set

In [6]:
def filterTrainSet(train_set):
    final_set = {}
    final_idx = 0
    
    for i in range(len(train_set)):
        data = {}
        if len(train_set[i]['input']) < len(train_set[i]['target']):
            continue
        else:
            data = train_set[i]
            final_set[final_idx] = data
            final_idx += 1
            
    return final_set

In [7]:
final_set = filterTrainSet(train_set)

In [8]:
lenTrainSet=len(final_set)
lenTrainSet

151228

#### Wordbank & Titlebank

In [9]:
class Word():
    def __init__(self):
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addTrainSet(self, train_set):
#        for word in sentence.split(' '):
#            self.addWord(word)
        for i in range(len(train_set)):
            for word in train_set[i]['target']:
                self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [10]:
class Title():
    def __init__(self):
        self.title2index = {}
        self.title2count = {}
        self.index2title = {0: "SOS", 1: "EOS"}
        self.n_titles = 2  # Count SOS and EOS

    def addTrainSet(self, train_set):
#        for word in sentence.split(' '):
#            self.addWord(word)
        for i in range(len(train_set)):
            for title in train_set[i]['input']:
                self.addTitle(title)

    def addTitle(self, title):
        if title not in self.title2index:
            self.title2index[title] = self.n_titles
            self.title2count[title] = 1
            self.index2title[self.n_titles] = title
            self.n_titles += 1
        else:
            self.title2count[title] += 1

In [11]:
wordbank=Word()
wordbank.addTrainSet(final_set)

In [12]:
titlebank=Title()
titlebank.addTrainSet(final_set)

#### Prepare Dataset

In [13]:
SOS_token = 0
EOS_token = 1

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
class PositionDataset(Dataset):
    
    def __init__(self,train_set):
        self.train_set=train_set
        
    def __len__(self):
        return len(self.train_set)
    
    def __getitem__(self,index):
        input_indexes=[]
        target_indexes=[]
        
        for item in self.train_set[index]['input']:
            input_indexes.append(titlebank.title2index[item])
        input_indexes.append(EOS_token)
        input_tensor = torch.tensor(input_indexes, dtype = torch.long, device = device).view(-1,1)
        
        for word in self.train_set[index]['target']:
            target_indexes.append(wordbank.word2index[word])
        target_indexes.append(EOS_token)
        while len(target_indexes) < len(input_indexes):
            target_indexes.append(EOS_token)
        target_tensor = torch.tensor(target_indexes, dtype = torch.long, device = device).view(-1)
        
        sample={'input':input_tensor, 'target':target_tensor}
        # input tensor: tensor(index1,index2...)
      
        return sample

## Model

### Positional Encoding

d_model = ninp (embedding dimension) <br>
nhid: the dimension of the feedforward network model in nn.TransformerEncoder <br>
nhead: the number of heads in the multiheadattention models  <br>
nlayers: the number of nn.TransformerEncoderLayer in nn.TransformerEncoder  <br>

In [16]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=100):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

#### questions: 
1. why the decoder is ntoken? so the input and output tokens are the same?
2. is ntokens here means the number of the words?

In [17]:
class TransformerModel(nn.Module):

    def __init__(self, n_titles, n_words, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropout) #get the positional encoder
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout) #get a layer
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers) # get the encoder
        self.encoder = nn.Embedding(n_titles, ninp) #embedding layer
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, n_words)

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask)
        output = self.decoder(output)
        return output

In [18]:
n_titles = titlebank.n_titles

In [19]:
n_titles

42302

In [20]:
n_words = wordbank.n_words

In [21]:
n_words

9186

## Initiate an instance

In [22]:
n_titles = titlebank.n_titles
n_words = wordbank.n_words
ninp = 200 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.2 # the dropout value

model = TransformerModel(n_titles, n_words, ninp, nhead, nhid, nlayers, dropout).to(device)

## Run the Model

### Preparations

In [23]:
import random

In [24]:
def chooseFromDataset(index):
    sample=positionsDataset[index]
    return (sample['input'],sample['target'])

positionsDataset = PositionDataset(final_set)

In [25]:
n_iters = 2000

In [26]:
training_pairs = [chooseFromDataset(random.randint(0,lenTrainSet-1)) for i in range(n_iters)]

In [27]:
evaluating_pairs = [chooseFromDataset(random.randint(0,lenTrainSet-1)) for i in range(n_iters)]

### Define train( ) and evaluate() 

In [28]:
criterion = nn.CrossEntropyLoss()
lr = 5.0 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

In [47]:
import time
def train():

    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    n_words = wordbank.n_words

    for iter_time in range(1, n_iters + 1):
        training_pair = training_pairs[iter_time - 1]  
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        seq_length = len(input_tensor)

        optimizer.zero_grad()
        output = model(input_tensor)
        output_flat = output.view(-1,n_words)
        
        loss = 0.
        for i in range(seq_length):
            loss_element = criterion(output_flat[i].view(-1,n_words),target_tensor[i].view(-1))
            loss += loss_element
            
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()/seq_length
        log_interval = 200
        if iter_time % log_interval == 0 and iter_time > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} iters | '
                  'lr {:02.2f} | ms/iter {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, iter_time, n_iters, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

In [48]:
def evaluate(eval_model, evaluating_pairs):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    n_words = wordbank.n_words

    with torch.no_grad():
        for iter_time in range(1, n_iters + 1):
            training_pair = training_pairs[iter_time - 1]
            input_tensor = training_pair[0]
            target_tensor = training_pair[1]
            seq_length = len(input_tensor)
        
            output = eval_model(input_tensor)
            output_flat = output.view(-1,n_words)
            
            loss = 0.
            for i in range(seq_length):
                loss_element = criterion(output_flat[i].view(-1,n_words),target_tensor[i].view(-1))
                loss += loss_element
            total_loss += loss.item()/seq_length
    return total_loss / len(evaluating_pairs)

### Loop over epochs

In [49]:
best_val_loss = float("inf")
epochs = 3 # The number of epochs
best_model = None

In [50]:
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    val_loss = evaluate(model, evaluating_pairs)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

    scheduler.step()

| epoch   1 |   200/ 2000 iters | lr 5.00 | ms/iter 63.72 | loss  4.50 | ppl    90.00
| epoch   1 |   400/ 2000 iters | lr 5.00 | ms/iter 64.18 | loss  4.54 | ppl    93.49
| epoch   1 |   600/ 2000 iters | lr 5.00 | ms/iter 72.81 | loss  4.44 | ppl    84.51
| epoch   1 |   800/ 2000 iters | lr 5.00 | ms/iter 64.19 | loss  4.60 | ppl    99.66
| epoch   1 |  1000/ 2000 iters | lr 5.00 | ms/iter 61.59 | loss  4.60 | ppl    99.58
| epoch   1 |  1200/ 2000 iters | lr 5.00 | ms/iter 61.11 | loss  4.54 | ppl    93.58
| epoch   1 |  1400/ 2000 iters | lr 5.00 | ms/iter 62.40 | loss  4.51 | ppl    90.69
| epoch   1 |  1600/ 2000 iters | lr 5.00 | ms/iter 62.44 | loss  4.35 | ppl    77.58
| epoch   1 |  1800/ 2000 iters | lr 5.00 | ms/iter 64.38 | loss  4.47 | ppl    87.47
| epoch   1 |  2000/ 2000 iters | lr 5.00 | ms/iter 62.57 | loss  4.36 | ppl    77.92
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 131.23s | valid loss  5

In [51]:
testing_pairs = [chooseFromDataset(random.randint(0,lenTrainSet-1)) for i in range(n_iters)]

In [52]:
test_loss = evaluate(best_model, testing_pairs)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)

| End of training | test loss  4.69 | test ppl   108.36


## Print out the results

In [53]:
n_print = 10

In [54]:
def evaluateFromDataset(index):
    each_set=final_set[index]
    sample=positionsDataset[index]
    return (each_set['input'],each_set['target'],sample['input'],sample['target'])

In [55]:
printing_pairs=[evaluateFromDataset(random.randint(0,lenTrainSet-1)) for i in range(n_print)]

In [56]:
def evalPrint(eval_model, input_tensor):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    n_words = wordbank.n_words

    with torch.no_grad():
        
        output = eval_model(input_tensor)
        output_flat = output.view(-1,n_words)
        softmax = nn.LogSoftmax(dim=1) 
        final = softmax(output_flat)
        decoded_words = []
        for i in range(final.size(0)):
            topv, topi = final[i].data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(wordbank.index2word[topi.item()])
                
    return decoded_words

In [57]:
def evaluateRandomly(model, n=10):
    for i in range(n):
        pair = printing_pairs[i]
        print('input sequence: ', pair[0])
        target_title = ' '.join(pair[1])
        print('target title: ', target_title)
        output_words = evalPrint(model, pair[2])
        output_title = ' '.join(output_words)
        print('output title: ', output_title)
        print('-'*80)

In [58]:
evaluateRandomly(best_model)

input sequence:  ['Executive, Corporate Affairs', 'Senior Marketing Executive, SG Operations Marketing Division', 'Team Leader, SG Fashion Promotion & Marketing']
target title:  Campaign Specialist
output title:  Director Director Director <EOS>
--------------------------------------------------------------------------------
input sequence:  ['Reporter @ Streats', 'Foreign Desk Journalist']
target title:  Sub-Editor
output title:  Director Director <EOS>
--------------------------------------------------------------------------------
input sequence:  ['Director']
target title:  Director
output title:  Director <EOS>
--------------------------------------------------------------------------------
input sequence:  ['Risk Manager and Internal Audit', 'AVP Business Control and Governance']
target title:  Vice President
output title:  Director Director <EOS>
--------------------------------------------------------------------------------
input sequence:  ['Executive Director', 'Group Execut

In [59]:
positionsDataset[3]['target']

tensor([4, 5, 1, 1])